In [ ]:
# !pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolo11l.pt')

In [ ]:
class_list = model.names

In [ ]:
# Open the video file
cap = cv2.VideoCapture('/content/4.mp4')

In [ ]:
from google.colab.patches import cv2_imshow

class_counts = defaultdict(int)
crossed_ids = set()
line_y_red = 430

while cap.isOpened():
  ret,frame = cap.read()
  if not ret:
    break

  result = model.track(frame,persist = True)

  if result[0].boxes.data is not None:
    boxes = result[0].boxes.xyxy.cpu()
    track_ids = result[0].boxes.id.int().cpu().tolist()
    class_indices = result[0].boxes.cls.int().cpu().tolist()
    confidences = result[0].boxes.conf.cpu()

    cv2.line(frame, (690, line_y_red),(1130, line_y_red),(0,0,255),3)
    # cv2.putText(frame, 'Red Line')


  for box,track_id,class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
    x1,y1,x2,y2 = map(int,box)

    cx = (x1 + x2)//2
    cy = (y1 + y2)//2

    class_name = class_list[class_idx]

    cv2.circle(frame, (cx, cy),4, (0,0,255),-1)
    cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255),2)
    cv2.rectangle(frame,(x1,y1),(x2,y2), (0,255,0),2)

    if cy > line_y_red and track_id not in crossed_ids:
      crossed_ids.add(track_id)
      class_counts[class_name] +=1

    # display the counts on the frame
    y_offset = 30
    for class_name, count in class_counts.items():
      cv2.putText(frame, f"{class_name}: {count}", (50,y_offset),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255, 0),2)

      y_offset+=30

    cv2_imshow(frame)
  if cv2.waitKey(0) & 0xFF == ord('q'):
    break

